# Use of the Ansible built-in URI method to query a Redfish service

Version 0.153

## Introduction

This Jupyter Notebook uses the Ansible built-in [`uri`](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/uri_module.html) method and an authentication token for getting and setting parameters of an iLO 5  Redfish service.

The use case of this Notebook is to explain how you can get and set the properties of a OneView managed compute node without any additional iLO user than the built-in `_HPOneViewAdmin` user.

In addition you will learn the basic methodology to discover Redfish resources instead of assuming their final location in the Redfish tree.

## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity to your Synegy iLO 5 simulator.

In [ ]:
########## Environment preparation (Version: 0.132) ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)
NbId=2
InvFile=${NbId}/hosts

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Fake Credentials as we are testing against a BMC simulator
OvSsoToken="FakeOvSsoToken"

# Miscellaneous
WorkshopDir=$PWD
HpePythonRedfishVenv="${NbId}/HpePythonRedfishVenv"
export PYTHONPATH="${WorkshopDir}/${NbId}/library/"
w=$(basename $PWD)

alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null ; sleep 1"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo -e "\n\tGood News: $bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port=${iLO5SimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}
# Below is a fake session token as we are testing against an iLO 5 simulator
token="${OvSsoToken}"
__EOF__


## Virtual Python environment creation

In order to completely isolate this notebook environment from other notebooks or student python environments, it is safer to create your dedicated Python virtual environment.

> NOTE: This `Venv` creation can take up to **2 minutes**. Just wait until the message `Finished creating Venv` is diplayed

In [ ]:
# Create Virtual Python environment (Venv)
[ -d ${HpePythonRedfishVenv} ] && rm -r ${HpePythonRedfishVenv} &>/dev/null 
python3 -m venv ${HpePythonRedfishVenv}                         &>/dev/null
source ${HpePythonRedfishVenv}/bin/activate                     &>/dev/null
export PS1="[PEXP\[\]ECT_PROMPT>"                               # Avoid Venv long prompt messing up outputs

# Install latest Ansible in the Venv
pip install jmespath                                            &>/dev/null
pip install ansible                                             &>/dev/null

echo -e "\n\n\tFinished creating Venv\n\n"

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart your iLO 5 simulator to restart this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart  (Version 0.1)
ResetSimulators 

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## Get and Set Redfish properties using Ansible built-in uri module 

In this section you will change the status of the Unit Identification light (UID, also called `IndicatorLED` in Redfish terminology) of compute nodes in order to facilitate their location in the Datacenter before maintenance. To make this exercise more realistic, you will automatically modify the enclosure UID/LED of these compute nodes if they are part of an enclosure/frame infrastructure.

The `IndicatorLED` resource location is standardized by Redfish as part of the `Chassis` data type and documented in the [HPE Redfish API reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#indicatorled):


![IndicatorLED Chassis property](Pictures/IndicatorLEDProperty.png)


The `Chassis` [data type](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#chassis-v1_6_0-chassis), as per Redfish, is located at `/redfish/v1/Chassis`. From this entry point you will retrieve the `{item}` list of each chassis composing this data type.

In a Synergy compute node, this list is composed of two chassis: a chassis called `enclosurechassis` containing the properties of the frame enclosure, and a chassis called `1` for the compute node.

> **NOTE**: The chassis names are not standadized by Redfish and may change over time.
Moreover, this naming convention, is definitively different for Moonshot, Superdome and other vendors of blade computers. Hence, if you want your script to work against other Redfish implementations than ilO based servers, you need to discover each `{item}` in the `Chassis` collection instead of assuming it.

![Chassis data type location](Pictures/ChassisDataType.png)


### Show, modify and verify a Redfish property using Ansible uri

The following cell discovers the `Chassis` collection of the nodes listed in the [hosts](hosts) inventory file created in the first cell of this notebook. Then, it prints selected properties of each member of the collection, including the location and the value of the `IndicatorLED`. 

The next tasks toggle the `IndicatorLED` value (`Off` - `Lit`) and apply the modification on each item of the collection. 

The last part of the playbook, validates the modification by retrieving again the `IndicatorLED` propety of each item of the chassis collection.

All of the above is performed using the [ansible.builtin.uri](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/uri_module.html) module and a (fake) OneView SSO token obtained in the previous notebook.



A convenient way to study the playbook of the next cell is to open it in a different view in this pane. Right click on this Notebook tab name and select `New View for Notebook` to open a new view:
<img src="Pictures/2-NewViewForNotebook.png" align=middle alt="New View for Notebook" height=30px />

Then, click on this [file link](2/SetIndicatorLEDUsingBuiltInUri.yml).

If you need more space, type `Ctrl-B` to hide the left pane. You can have it reappear by hitting `Ctrl-B` again.

In [ ]:
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingBuiltInUri.yml

# ****  NEED To Decide wether the following is relevant in this workshop ****

**Pro**: the Security Dashboard parameters is an HPE/Oem extension. This example can show how chiant it is to crawl and discover all of them using the Ansible `uri`

**Cons**: May be too long for 105 minutes ? May be too long and too chiant to develop ?




## Enforce or ignore items in the security dashboard

The ilO 5 security dashboard lists the security parameters at risk. Each parameter can be enforced or ignored. OneView, as of the writing of this workshop, does not offer the possibility to modify those settings. 

In this paragraph, you will learn how to retrieve the status of all the security dasboard parameters and then enforce one of them TBD

In [ ]:
ansible-playbook -i ${NbId}/hosts ${NbId}/GetSecurityParameters.yml

### Set ignore = False

In order to tbd

In [ ]:
ansible-playbook -i ${NbId}/hosts ${NbId}/SetSecurityParamUsingBuilInUri.yml

# ****  END of NEED To Decide wether the following is relevant in this workshop ****

## Summary

In this workshop, you used the Ansible build-in `uri` module to get and set Redfish properties, using a OneView Single Sign On (fake) token and without assuming their location. 


Select the next [Notebook](3-RedfishAnsibleUsingHpePlaybooks.ipynb) to perform the same exercise with the HPE provided playbooks.